In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

In [53]:
import matplotlib.pyplot as plt, argparse, numpy as np, math, sys, copy
from skimage.segmentation import slic, mark_boundaries, felzenszwalb
from skimage.util import img_as_float
from skimage import io
import numpy as np
import superpixels as sp
import glob
import pipeline as pipe
import time
import superpixels as sp
import scipy.stats as sci
from PIL import Image
from scipy import ndimage
import cv2

In [75]:
ground_path = 'BSDS500/data/groundTruth/val/'
image_path = 'BSDS500/data/images/val/'
save_path = 'BSDS500/pre_process/images/val/'
bound_prefix = 'images/'
eval_prefix = 'eval/'

epsilon = 0.0001 #just to round down
DEFAULT_IMG_SIZE = (481,481,3) #defaul image size

### Read Images

In [13]:
len_path = len(image_path)
image_files = glob.glob(image_path + '*.jpg')
image_filenames = [x[len_path:-4] for x in image_files]

image_names, names = [], []

#load images
for filename in image_filenames:
    file = image_path + filename + '.jpg'
    
    #read groundtruth and image
    #image = img_as_float(io.imread(train_file))
    
    #append image and groundtruths
    image_names.append(file)
    names.append(filename + '.png')
        
print(image_names[0])

BSDS500/data/images/val/62096.jpg


In [82]:
index = 0
len_images = len(names)
for image_name, name in zip(image_names, names):
    index += 1

    black_image = np.zeros(DEFAULT_IMG_SIZE,dtype=np.uint8)
    image = io.imread(image_name)
    
    prefix_name = name[:-4]
    extension_name = name[-4:]

    if(image.shape[0] == 321):
        black_image[80:401,:,:] = image[:,:,:]
    else:
        black_image[:,80:401,:] = image[:,:,:]

    #save images
    save_name = (save_path + prefix_name + '_noflip_0' + extension_name)
    io.imsave(save_name, black_image)

    for angle in range(30, 360, 30):
        rot_image = ndimage.rotate(image, angle)
        res_image = cv2.resize(rot_image, dsize=DEFAULT_IMG_SIZE[:2], interpolation=cv2.INTER_CUBIC)

        save_name = (save_path + prefix_name + '_noflip_' + str(angle) + extension_name)
        io.imsave(save_name, res_image)
        
    print(index, '/', len_images, end='')
    print('\r', end='')  

#images.append(np.fliplr(left_image))
print('OK!')

OK! / 100


### Generate boundaries and save results

In [19]:
ini = 0
fim = len(ground_filenames)

methods = ['hsgb'] #['hslic', 'hsgb', 'hegb']

for method in methods:
    last_shape = None
    blank_image = None
    index = 0
    
    starttime = time.time()
    for image, name in zip(images[ini:fim], names[ini:fim]):
        if(image.shape != last_shape):
            blank_image = np.zeros(image.shape,dtype=np.uint8) #create blank image to save
            blank_image.fill(255)
            last_shape = image.shape
        
        ##generate ultrametric map
        ultra_image, cluster_sizes = pipe.generate_ultrametric_image(image, blank_image, method)
        
        #convert to grayscale
        gray_image = np.dot(ultra_image[...,:3], [0.299, 0.587, 0.114])
        
        #save image
        boundname = save_path + method + '/' + bound_prefix + name
        io.imsave(boundname, gray_image)
        
        #verbose mode
        index += 1
        if index % 10 == 0:
            print(index, cluster_sizes)

    endtime = time.time()
        
    #np.savez(boundname, borders=borders)
    print((endtime - starttime), 'seconds')

10 [40, 32, 24, 16, 8]
20 [40, 32, 24, 16, 8]
30 [40, 32, 24, 16, 8]
40 [40, 32, 24, 16, 8]
50 [40, 32, 24, 16, 8]
60 [40, 32, 24, 16, 8]
70 [40, 32, 24, 16, 8]
80 [40, 32, 24, 16, 8]
90 [40, 32, 24, 16, 8]
100 [40, 32, 24, 16, 8]
409.91629433631897 seconds


### Recover boundaries and evaluate results

In [20]:
ini = 0
fim = len(ground_filenames)

methods = ['hsgb'] #['hslic', 'hsegb', 'hegb']

for method in methods:    
    threshold = 0
    index = 0

    starttime = time.time()
    
    for groundtruth, name in zip(groundtruths[ini:fim], names[ini:fim]):
        precisions, recalls = [], []
        
        boundname = save_path + method + '/' + bound_prefix + name
        evalname = save_path + method + '/' + eval_prefix + name + extension
        
        image = img_as_float(io.imread(boundname))        
        unique = np.unique(image)
        
        #adjust number of clusters (only to some itens)
        if(len(unique) <= number_clusters):
            len_unique = len(unique)
            for i in range(number_clusters-len_unique+1):
                unique = np.append(unique, unique[len_unique-1])
        
        for thresh in unique[1:]:
            thresh_image = sci.threshold(image, (thresh - epsilon))

            #calculate values from precision and recall
            precision, recall = ev.eval_bound(groundtruth[:,:,0], thresh_image, threshold, True)
            precisions.append(precision)
            recalls.append(recall)
        
        #verbose mode
        index += 1
        if index % 10 == 0:
            print(index)
        
        #save precision file
        np.savez(evalname, precisions=precisions, recalls=recalls)
            
    endtime = time.time()
    print((endtime - starttime), 'seconds')

10
20
30
40
50
60
70
80
90
100
169.21013498306274 seconds


### Load evaluations and calculate f-measures

In [22]:
ini = 0
fim = len(ground_filenames)

methods = ['hsgb'] #['hslic', 'hsegb', 'hegb']

for method in methods:
    precisions, recalls = [], []
    fmeasures = []
    
    for name in names[ini:fim]:
        evalname = save_path + method + '/' + eval_prefix + name + extension
        npload = np.load(evalname)
        precisions.append(npload['precisions'])
        recalls.append(npload['recalls'])

    for i in range(0,len(precisions[0])):
        cutz_precision, cutz_recall = [], []
        
        for precision, recall in zip(precisions, recalls):
            cutz_precision.append(precision[i])
            cutz_recall.append(recall[i])
    
        np_precisions = (np.array(cutz_precision))
        np_recalls = (np.array(cutz_recall))

        fmeasure = 2 * ((np_precisions * np_recalls) / (np_precisions + np_recalls))
        fmeasures.append(fmeasure)
    
    #print(fmeasures)
print(np.average(fmeasures, axis=1))

[ 0.15525036  0.20788418  0.2280259   0.23445528  0.23834142]
